![横幅](../static/imgs/MagicMovieMachine_banner.png)
# 构建您自己的电影推荐器

现在，是时候在 AWS 账户中构建您自己的电影推荐器了。在本 Jupyter 笔记本中，
您将看到如何创建推荐器来生成推荐，就像 Magic Movie Machine 一样！

您将首先下载、修改从 [MovieLens 项目](https://grouplens.org/datasets/movielens/)数据集收集的电影数据，然后将数据导入到 Amazon Personalize 中。然后，您将让 Amazon Personalize 使用此数据创建推荐器，一个用于*精品甄选*使用案例，另一个用于*更像 X*使用案例。

这些是生成推荐的预配置使用案例。在幕后，Amazon Personalize 为每个推荐器训练多个使用案例特定模型。*精品甄选*生成的推荐与您在 Magic Movie Machine 中看到的最佳推荐相似。*更像 X*是*隐藏的瑰宝*推荐的更简单版本：它为类似电影生成推荐，但它们并不是鲜为人知的瑰宝。

最后，您将获得电影推荐，就像您在 Magic Movie Machine 中看到的一样！

## 教程成本和持续时间
完成本教程可能需要 2 小时左右。这可能会产生一些成本，具体取决于您的 AWS 账户。如果您使用的是 AWS 免费套餐，请在开始的当天完成教程，并在完成后删除所有资源，成本将近乎为零。有关收费和价格的完整列表，请参阅 [Amazon Personalize 定价](https://aws.amazon.com/personalize/pricing/)。

为避免产生不必要的费用，请确保运行在此存储库中找到的 `Clean_Up_Resources.ipynb` 笔记本。
# 如何使用笔记本

此代码被分解为如下所示的单元格。在本页顶部有一个三角形 Run（运行）按钮，您可以单击该按钮以执行每个单元格，并移至下一个单元格；也可以按 `Shift` + `Enter` 组合键，在单元格中执行并移至下一个单元格。

当一个单元格执行时，您会注意到该单元格运行时旁边有一行显示 `*`，或者它会更新为一个数字，以指示在执行完单元格中的所有代码后完成执行的最后一个单元格。

只需按照下面的说明执行单元格，即可通过案例优化推荐器开始使用 Amazon Personalize。

## 导入
Python 附带有许多库。您需要导入这些库和其他文件包，如 [boto3](https://aws.amazon.com/sdk-for-python/)（适用于 Python 的 AWS SDK）和 [Pandas](https://pandas.pydata.org/)/[Numpy](https://numpy.org/)，它们是核心数据科学工具。

In [1]:
# Imports
import boto3
import json
import numpy as np
import pandas as pd
import time
import datetime

接下来，您应该验证您的环境是否可以成功地与 Amazon Personalize 进行通信：

In [2]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

## 下载、准备和上载训练数据
本笔记本使用从 [MovieLens 项目](https://grouplens.org/datasets/movielens/)收集的交互数据。执行以下行，下载最新版本的 MovieLens 数据，并将数据导入到本笔记本中，然后快速检查。

### 下载并浏览数据集

In [9]:
!wget -N https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
!unzip -o ml-latest-small.zip

--2022-04-09 16:48:16--  https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Resolving files.grouplens.org... 128.101.65.152
Connecting to files.grouplens.org|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 304 Not Modified
File 'ml-latest-small.zip' not modified on server. Omitting download.

Archive:  ml-latest-small.zip
  inflating: ml-latest-small/links.csv  
  inflating: ml-latest-small/tags.csv  
  inflating: ml-latest-small/ratings.csv  
  inflating: ml-latest-small/README.txt  
  inflating: ml-latest-small/movies.csv  


In [5]:
!ls ml-latest-small

README.txt  links.csv   movies.csv  ratings.csv tags.csv


In [11]:
!pygmentize ml-latest-small/README.txt

Summary

This dataset (ml-latest-small) describes 5-star rating and free-text tagging activity from [MovieLens](http://movielens.org), a movie recommendation service. It contains 100836 ratings and 3683 tag applications across 9742 movies. These data were created by 610 users between March 29, 1996 and September 24, 2018. This dataset was generated on September 26, 2018.

Users were selected at random for inclusion. All selected users had rated at least 20 movies. No demographic information is included. Each user is represented by an id, and no other information is provided.

The data are contained in the files `links.csv`, `movies.csv`, `ratings.csv` and `tags.csv`. More details about the contents and use of all these files follows.

This is a *development* dataset. As such, it may change over time and is not an appropriate dataset for shared research results. See available *benchmark* datasets if that is your intent.

This and other GroupLens data sets are publicly available for down

In [7]:
interactions_data = pd.read_csv('./ml-latest-small/ratings.csv')
pd.set_option('display.max_rows', 5)
interactions_data

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
...,...,...,...,...
100834,610,168252,5.0,1493846352
100835,610,170875,3.0,1493846415


In [8]:
interactions_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


## 准备数据

### 交互数据
正如您所看到的，交互数据包含 UserID、ItemID、Rating 和 Timestamp。导入数据之前，您需要完成几个步骤。

运行以下代码，删除排名较低的项目，并在导入数据之前删除 Rating 列。您还可以为每个交互添加“观看”事件类型的列 EVENT_TYPE。

In [ ]:
interactions_data = interactions_data[interactions_data['rating'] > 3]                # Keep only movies rated higher than 3 out of 5.
interactions_data = interactions_data[['userId', 'movieId', 'timestamp']]
interactions_data.rename(columns = {'userId':'USER_ID', 'movieId':'ITEM_ID', 
                              'timestamp':'TIMESTAMP'}, inplace = True)
interactions_data['EVENT_TYPE']='watch' #Adds an EVENT_TYPE column and an event type of "watch" for each interaction.
interactions_data.head()

In [ ]:
items_data = pd.read_csv('./ml-latest-small/movies.csv')
items_data.head(5)

### 项目元数据

要尽可能创建最佳的 Magic Movie Machine，您还需要上载每部电影的相关数据。打开项目数据文件并查看第一行。

In [ ]:
items_data = pd.read_csv('./ml-latest-small/movies.csv')
items_data.head(5)

In [ ]:
items_data = pd.read_csv('./ml-latest-small/movies.csv')
items_data.head(5)
items_data.info()

In [ ]:
items_data['year'] = items_data['title'].str.extract('.*\((.*)\).*',expand = False)
items_data.head(5)

由于每部电影的实际创建时间戳未知，因此下列电影添加了一个现代日期作为创建时间戳。

In [ ]:
ts= datetime.datetime(2022, 1, 1, 0, 0).strftime('%s')
print(ts)

In [ ]:
items_data["CREATION_TIMESTAMP"] = ts
items_data

In [ ]:
# removing the title
items_data.drop(columns="title", inplace = True)

# renaming the columns to match schema
items_data.rename(columns = { 'movieId':'ITEM_ID', 'genres':'GENRES',
                              'year':'YEAR'}, inplace = True)
items_data

### 用户元数据

要使 Magic Movie Machine 基于用户元数据提供个性化数据，您需要导入用户的相关数据。MovieLens 数据集没有任何用户元数据，因此您可以为与电影交互的每个用户创建假元数据，如下所示：

In [ ]:
# get user ids from the interaction dataset

user_ids = interactions_data['USER_ID'].unique()
user_data = pd.DataFrame()
user_data["USER_ID"]=user_ids
user_data

#### 添加元数据
当前数据集不包含其他用户信息。在本示例中，我们将为用户随机分配一个性别，且男女比例相同。这些属性可以产生更个性化的推荐。

In [ ]:
possible_genders = ['female', 'male']
random = np.random.choice(possible_genders, len(user_data.index), p=[0.5, 0.5])
user_data["GENDER"] = random
user_data

## 配置 S3 存储桶和 IAM 角色

到目前为止，我们已下载、操作数据并将数据保存到运行此 Jupyter 笔记本的 SageMaker 实例所连接的 Amazon EBS 实例中。但是，Amazon Personalize 需要一个 S3 存储桶充当您的数据源以及用于访问该存储桶的 IAM 角色。让我们把一切都安排好。

Amazon S3 存储桶需要与我们目前创建的 Amazon Personalize 资源位于同一区域。只需在下面将区域定义为字符串即可。

In [ ]:
# Sets the same region as current Amazon SageMaker Notebook
with open('/opt/ml/metadata/resource-metadata.json') as notebook_info:
    data = json.load(notebook_info)
    resource_arn = data['ResourceArn']
    region = resource_arn.split(':')[3]
print('region:', region)

# Or you can specify the region where your bucket and model will be domiciled this should be the same region as the Amazon Personalize resources
# region = "us-east-1"


In [ ]:
s3 = boto3.client('s3')
account_id = boto3.client('sts').get_caller_identity().get('Account')
bucket_name = account_id + "-" + region + "-" + "personalizemanagedvod"
print('bucket_name:', bucket_name)

try: 
    if region == "us-east-1":
        s3.create_bucket(Bucket=bucket_name)
    else:
        s3.create_bucket(
            Bucket=bucket_name,
            CreateBucketConfiguration={'LocationConstraint': region}
            )
except:
    print("Bucket already exists. Using bucket", bucket_name)

### 将数据上载到 S3
现在，您的 Amazon S3 存储桶已经创建，请上载包含我们的用户-项目-交互数据的 CSV 文件。

In [ ]:
interactions_filename = "interactions.csv"
interactions_data.to_csv(interactions_filename, index=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(interactions_filename).upload_file(interactions_filename)

items_filename = "items.csv"
items_data.to_csv(items_filename, index=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(items_filename).upload_file(items_filename)

user_filename = "users.csv"
user_data.to_csv(user_filename, index=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(user_filename).upload_file(user_filename)

## 设置 S3 存储桶策略
Amazon Personalize 需要能够读取 S3 存储桶的内容。因此，我们添加可实现此操作的存储桶策略。

注意：确保您用于在此笔记本中运行代码的角色具有修改 S3 存储桶策略所需的权限。

In [ ]:
s3 = boto3.client("s3")
policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:GetObject",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket_name),
                "arn:aws:s3:::{}/*".format(bucket_name)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket_name, Policy=json.dumps(policy))

## 创建并等待数据集组
现在是时候创建您的 Amazon Personalize 资源了。首先，您将创建一个域数据集组，域设置为 VIDEO_ON_DEMAND。域数据集组是一个容器，存储不同业务领域和使用案例的预配置资源，如数据集和推荐器。您可以随意更改下面的名称。

### 创建数据集组

In [ ]:
response = personalize.create_dataset_group(
    name='personalize-video-on-demand-ds-group',
    domain='VIDEO_ON_DEMAND'
)

dataset_group_arn = response['datasetGroupArn']
print(json.dumps(response, indent=2))

等待数据集组状态变为 ACTIVE。
在我们可以在以下任何项目中使用数据集组之前，数据集组必须处于活跃状态。执行下面的单元格，并等待单元格显示“ACTIVE”状态。

In [ ]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

## 创建交互模式
Personalize 如何理解您的数据的核心组件来自下面定义的模式。此配置指导服务如何读取通过 CSV 文件提供的数据。注意列和类型与您在上面创建的文件中的内容对齐。

In [ ]:
schema = {
  "type": "record",
  "name": "Interactions",
  "namespace": "com.amazonaws.personalize.schema",
  "fields": [
      {
          "name": "USER_ID",
          "type": "string"
      },
      {
          "name": "ITEM_ID",
          "type": "string"
      },
      {
          "name": "EVENT_TYPE",
          "type": "string"
      },
      {
          "name": "TIMESTAMP",
          "type": "long"
      }
  ],
  "version": "1.0"
}

create_interactions_schema_response = personalize.create_schema(
    name='personalize-demo-interactions-schema',
    schema=json.dumps(schema),
    domain='VIDEO_ON_DEMAND'
)

interactions_schema_arn = create_interactions_schema_response['schemaArn']
print(json.dumps(create_interactions_schema_response, indent=2))

# 创建项目（电影）架构

与之前类似，我们将为项目创建一个架构，在本例中为电影。

In [ ]:
schema = {
  "type": "record",
  "name": "Items",
  "namespace": "com.amazonaws.personalize.schema",
  "fields": [
    {
      "name": "ITEM_ID",
      "type": "string"
    },
    {
      "name": "GENRES",
      "type": [
        "string"
      ],
      "categorical": True
    },
    {
      "name": "YEAR",
      "type": [
        "string"
      ],
      "categorical": True
    }, 
    {
      "name": "CREATION_TIMESTAMP",
      "type": "long"
    }
  ],
  "version": "1.0"
}
create_items_schema_response = personalize.create_schema(
    name='personalize-demo-items-schema',
    schema=json.dumps(schema),
    domain='VIDEO_ON_DEMAND'
)

items_schema_arn = create_items_schema_response['schemaArn']
print(json.dumps(create_items_schema_response, indent=2))

# 创建用户架构

现在，我们为用户数据创建一个模式。

In [ ]:
schema = {
    "type": "record",
    "name": "Users",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
      {
          "name": "USER_ID",
          "type": "string"
      },
      {
          "name": "GENDER",
          "type": "string",
          "categorical": True
      }
    ],
    "version": "1.0"
}
create_users_schema_response = personalize.create_schema(
    name='personalize-demo-users-schema',
    schema=json.dumps(schema),
    domain='VIDEO_ON_DEMAND'
)

users_schema_arn = create_users_schema_response['schemaArn']
print(json.dumps(create_users_schema_response, indent=2))

## 创建数据集
下一个任务是使用您刚刚创建的模式，在您的域数据集组中创建实际的 Amazon Personalize 数据集。数据集将存储您的数据用于训练。

### 创建交互数据集

In [ ]:
dataset_type = "INTERACTIONS"

create_dataset_response = personalize.create_dataset(
    name = "personalize-demo-interactions",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = interactions_schema_arn
)

interactions_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

### 创建项目数据集

In [ ]:
dataset_type = "ITEMS"
create_dataset_response = personalize.create_dataset(
    name = "personalize-demo-items",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = items_schema_arn
)

items_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

### 创建用户数据集

In [ ]:
dataset_type = "USERS"
create_dataset_response = personalize.create_dataset(
    name = "personalize-demo-users",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = users_schema_arn
)

users_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

## 创建 Personalize 角色
Amazon Personalize 需要能够承担 AWS 中的角色，以便拥有执行某些任务的权限。以下各行授予此权限。

注意：确保您用于在此笔记本中运行代码的角色具有创建角色所需的权限。如果卡住，请参阅 [Amazon Personalize 开发人员指南](https://docs.aws.amazon.com/personalize/latest/dg/aws-personalize-set-up-permissions.html)。

In [ ]:
iam = boto3.client("iam")

role_name = "PersonalizeRoleVODDemoRecommender"
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}

create_role_response = iam.create_role(
    RoleName = role_name,
    AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
)

# AmazonPersonalizeFullAccess provides access to any S3 bucket with a name that includes "personalize" or "Personalize" 
# if you would like to use a bucket with a different name, please consider creating and attaching a new policy
# that provides read access to your bucket or attaching the AmazonS3ReadOnlyAccess policy to the role
policy_arn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
iam.attach_role_policy(
    RoleName = role_name,
    PolicyArn = policy_arn
)

# Now add S3 support
iam.attach_role_policy(
    PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess',
    RoleName=role_name
)
time.sleep(60) # wait for a minute to allow IAM role policy attachment to propagate

role_arn = create_role_response["Role"]["Arn"]
print(role_arn)


## 导入数据
之前，您创建了域数据集组和数据集，以存储 Magic Movie Machine 数据。现在，您将执行一个导入作业，将 S3 中的数据加载到 Amazon Personalize 中，用以构建您的电影推荐器。
### 创建交互数据集导入作业

In [ ]:
create_interactions_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "personalize-demo-import-interactions",
    datasetArn = interactions_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, interactions_filename)
    },
    roleArn = role_arn
)

dataset_interactions_import_job_arn = create_interactions_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_interactions_dataset_import_job_response, indent=2))

### 创建项目数据集导入作业

In [ ]:
create_items_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "personalize-demo-import-items",
    datasetArn = items_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, items_filename)
    },
    roleArn = role_arn
)

dataset_items_import_job_arn = create_items_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_items_dataset_import_job_response, indent=2))

### 创建用户数据集导入作业

In [ ]:
create_users_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "personalize-demo-import-users",
    datasetArn = users_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, user_filename)
    },
    roleArn = role_arn
)

dataset_users_import_job_arn = create_users_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_users_dataset_import_job_response, indent=2))

等待数据集导入作业状态变为 ACTIVE。
导入作业可能需要一会儿才能完成，请等待直至看到它在下面处于活跃状态。

In [ ]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_interactions_import_job_arn
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print("Interactions DatasetImportJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)
    
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_items_import_job_arn
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print("Items DatasetImportJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)
    
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_users_import_job_arn
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print("Users DatasetImportJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

## 选择推荐器使用案例

每个域都有不同的使用案例。您需要为特定使用案例创建推荐器，并且每个使用案例获得推荐的要求都不同。

有关 Amazon Personalize 支持的所有使用案例的信息，请参阅[选择推荐使用案例](https://docs.aws.amazon.com/personalize/latest/dg/domain-use-cases.html)。

In [ ]:
available_recipes = personalize.list_recipes(domain='VIDEO_ON_DEMAND') # See a list of recommenders for the domain. 
if (len(available_recipes["recipes"])==0):
    # This is a workaround to get the recipes in case 'available_recipes["recipes"]'does not retrieve them
    available_recipes = personalize.list_recipes(domain='VIDEO_ON_DEMAND', nextToken=available_recipes["nextToken"])
display(available_recipes["recipes"])
    

我们将创建一个类型为*更像 X*的推荐器。*更像 X*是您在 Magic Movie Machine 中看到的*隐藏的瑰宝*推荐的更简单版本：它为类似电影生成推荐，但它们并不是鲜为人知的瑰宝。对于此使用案例，Amazon Personalize 会根据 `get_recommendations` 调用中指定的用户 ID 自动筛选用户观看过的视频。

In [ ]:
create_recommender_response = personalize.create_recommender(
  name = 'more_like_x_demo',
  recipeArn = 'arn:aws:personalize:::recipe/aws-vod-more-like-x',
  datasetGroupArn = dataset_group_arn
)
recommender_more_like_x_arn = create_recommender_response["recommenderArn"]
print (json.dumps(create_recommender_response))

我们将创建第二个类型为"精品甄选"的推荐器。这种类型的推荐器为您指定的用户提供个性化流媒体内容推荐。在此使用案例中，Amazon Personalize 会根据您指定的用户 ID 和 `Watch` 事件自动筛选用户已观看的视频。

In [ ]:
create_recommender_response = personalize.create_recommender(
  name = 'top_picks_for_you_demo',
  recipeArn = 'arn:aws:personalize:::recipe/aws-vod-top-picks',
  datasetGroupArn = dataset_group_arn
)
recommender_top_picks_arn = create_recommender_response["recommenderArn"]
print (json.dumps(create_recommender_response))

我们等待直至推荐器完成创建且状态变为 `ACTIVE`。我们定期检查推荐器的状态

In [ ]:
%%time

max_time = time.time() + 10*60*60 # 10 hours
while time.time() < max_time:

    version_response = personalize.describe_recommender(
        recommenderArn = recommender_more_like_x_arn
    )
    status = version_response["recommender"]["status"]

    if status == "ACTIVE":
        print("Build succeeded for {}".format(recommender_more_like_x_arn))
        
    elif status == "CREATE FAILED":
        print("Build failed for {}".format(recommender_more_like_x_arn))

    if status == "ACTIVE":
        break
    else:
        print("The More Like X Recommender build is still in progress")
        
    time.sleep(60)
    
while time.time() < max_time:

    version_response = personalize.describe_recommender(
        recommenderArn = recommender_top_picks_arn
    )
    status = version_response["recommender"]["status"]

    if status == "ACTIVE":
        print("Build succeeded for {}".format(recommender_top_picks_arn))
        
    elif status == "CREATE FAILED":
        print("Build failed for {}".format(recommender_top_picks_arn))

    if status == "ACTIVE":
        break
    else:
        print("The Top Pics for You Recommender build is still in progress")
        
    time.sleep(60)

## 获得推荐
现在，您已经创建了电影推荐器，我们来看看您可以为用户提供的电影推荐！

In [ ]:
# reading the original data in order to have a dataframe that has both movie_ids 
# and the corresponding titles to make out recommendations easier to read.
items_df = pd.read_csv('./ml-latest-small/movies.csv')
items_df.sample(10)

In [ ]:
def get_movie_by_id(movie_id, movie_df):
    """
    This takes in an movie_id from a recommendation in string format,
    converts it to an int, and then does a lookup in a specified
    dataframe.
    
    A really broad try/except clause was added in case anything goes wrong.
    
    Feel free to add more debugging or filtering here to improve results if
    you hit an error.
    """
    try:
        return movie_df.loc[movie_df["movieId"]==int(movie_id)]['title'].values[0]
    except:
        print (movie_id)
        return "Error obtaining title"

### 获得“更像 X”的电影推荐

“更像 X”推荐器为类似电影生成推荐。您提供电影 ID，Amazon Personalize 将返回类似电影清单。

In [ ]:
# First pick a user
test_user_id = "1"

# Select a random item
test_item_id = "81847" #Iron Man 59315, Tangled: 81847

# Get recommendations for the user for this item
get_recommendations_response = personalize_runtime.get_recommendations(
    recommenderArn = recommender_more_like_x_arn,
    userId = test_user_id,
    itemId = test_item_id,
    numResults = 20
)

# Build a new dataframe for the recommendations
item_list = get_recommendations_response['itemList']
recommendation_list = []
for item in item_list:
    movie = get_movie_by_id(item['itemId'], items_df)
    recommendation_list.append(movie)

user_recommendations_df = pd.DataFrame(recommendation_list, columns = [get_movie_by_id(test_item_id, items_df)])

pd.options.display.max_rows = 20
display(user_recommendations_df)

### 获取“精品甄选”推荐

现在，您将通过*精品甄选*推荐器为用户提供个性化推荐。首先，使用以下代码将用户的元数据添加到我们的示例用户。您可以使用此类型的元数据获取有关用户的见解。

In [ ]:
users_data_df = pd.read_csv('./users.csv')

def get_gender_by_id(user_id, user_df):
    """
    This takes in a user_id and then does a lookup in a specified
    dataframe.
    
    A really broad try/except clause was added in case anything goes wrong.
    
    Feel free to add more debugging or filtering here to improve results if
    you hit an error.
    """
    return user_df.loc[user_df["USER_ID"]==int(user_id)]['GENDER'].values[0]
    try:
        return user_df.loc[user_df["USER_ID"]==int(user_id)]['GENDER'].values[0]
    except:
        print (user_id)
        return "Error obtaining title"

接下来，挑选一个用户并为他们提供电影推荐。

In [ ]:
# First pick a user
test_user_id = "111" # samples users: 55, 75, 76, 111

# Get recommendations for the user
get_recommendations_response = personalize_runtime.get_recommendations(
    recommenderArn = recommender_top_picks_arn,
    userId = test_user_id,
    numResults = 20
)

# Build a new dataframe for the recommendations
item_list = get_recommendations_response['itemList']
recommendation_list = []
for item in item_list:
    movie = get_movie_by_id(item['itemId'], items_df)
    recommendation_list.append(movie)

column_name = test_user_id+" ("+get_gender_by_id(test_user_id, users_data_df)+")"

user_recommendations_df = pd.DataFrame(recommendation_list, columns = [column_name])

pd.options.display.max_rows =20
display(user_recommendations_df)

## 审核
使用上述代码，您成功创建了如 Magic Movie Machine 中一样工作的推荐器：您训练了一个深度学习模型，以根据之前的用户行为生成电影推荐。

您为两个基本使用案例创建了两个推荐器：一个用于*精品甄选*使用案例，另一个用于*更像 X*使用案例。接下来，您可以调整此代码以创建其他推荐器。

## 适用于下一个笔记本的注意事项：
您将需要一些值以用于下一个笔记本，请执行下面的单元格来存储这些值，以便在 `Clean_Up_Resources.ipynb` 笔记本中加以使用。

这将覆盖为这些变量存储的任何数据，并将数据设置为此笔记本中指定的值。 

In [ ]:
# store for cleanup
%store dataset_group_arn
%store role_name
%store region